In [1]:
import retire_nonconvex as retire
import numpy as np
import numpy.random as rgt
import pandas as pd

In [2]:
n, p = 200, 10
itcp, beta, true_loc = 2, np.zeros(p), np.zeros(p)
beta[:10] = [1.8,  1.6,  1.4,  1.2,  1,  -1,  -1.2, -1.4, -1.6, -1.8] #s = 10
true_active_set = np.where(beta!=0)[0]  #true_active_set is of size s
true_loc[true_active_set] = 1           #true_loc is of size p, with only s entries = 1 and p-s entries = 0

cov_matrix = np.zeros((p,p))
for i in range(p):
    for j in range(p):
        cov_matrix[i,j] = 0.5**abs(i-j)

In [3]:
def sim(tau_input=0.5, quantile=0, expectile=0, M=10, robust=(n/np.log(n*p))**0.5,\
        noise_type='normal', model_type='model1'):
    print('model_type=',model_type, ' noise_type=',noise_type, ' robust=',robust)
    print("tau=",tau_input, ' expectile=',expectile, ' quantile=',quantile, ' M=', M, '\n')
    l2loss_vec_retire = np.zeros(M)
    for m in range(M):
        rgt.seed(100+m)
        X = rgt.multivariate_normal(np.zeros(p), cov_matrix, size = n)
        
        if noise_type == 'normal':
            noise = rgt.normal(0,2**0.5,n) #sd=sqrt(2)
        elif noise_type == 't':
            noise = rgt.standard_t(2.1,n)
        else:
            raise ValueError("noise_type must be either normal, t")
            
        if model_type == 'model1':
            Y = itcp + X.dot(beta) + noise                                         #Homoscedastic 
        elif model_type == 'model2':
            Y = itcp + X.dot(beta) + (0.5 + 0.5*abs(X[:,-1]))*(noise - quantile)   #Quantile Hetero
        elif model_type == 'model3':
            Y = itcp + X.dot(beta) + (0.5 + 0.5*abs(X[:,-1]))*(noise - expectile)  #Expectile Hetero
        else:
            raise ValueError("model_type must be either model1, model2, model3")
            
        temp = retire.low_dim(X, Y)
        result_temp_retire = temp.fit(tau_input,robust=robust, standardize=True, adjust=True)   
        l2loss_retire = (np.sum((result_temp_retire['beta'][1:] - beta)**2) + (result_temp_retire['beta'][0]-itcp)**2)**0.5 
        l2loss_vec_retire[m] = l2loss_retire    
        
    data_ld = {'l2loss_reitre':l2loss_vec_retire}
    df = pd.DataFrame(data_ld)
    return(df)

In [4]:
M=10                                #number of repetition
robust = (n/(p + np.log(n)))**0.5   #(n/np.log(n*p))**0.5 or False [lowdim robust]
model_type = 'model3'               #'model1' or 'model2' or 'model3'
tau_input = 0.5                     #0.5 or 0.8
noise_type = 'normal'               #'normal' or 't'
quantile = 0                        #depends on noise type and tau level
expectile = 0.0                     #depends on noise type and tau level

In [5]:
df = sim(tau_input, quantile, expectile, M, robust, noise_type, model_type)
df

model_type= model3  noise_type= normal  robust= 3.6157064563212673
tau= 0.5  expectile= 0.0  quantile= 0  M= 10 



,l2loss_reitre
0,0.289138
1,0.278213
2,0.269231
3,0.441292
4,0.367157
5,0.405818
6,0.414273
7,0.529930
8,0.508145
9,0.259511


In [6]:
print(df.mean())
print(df.std())

l2loss_reitre    0.376271
dtype: float64
l2loss_reitre    0.09996
dtype: float64
